## Import library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score # モデル評価用(正答率)
from sklearn.metrics import log_loss # モデル評価用(logloss)     
from sklearn.metrics import roc_auc_score # モデル評価用(auc)
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_columns',100)
pd.set_option('Display.max_rows',100)

## Import Dataset

In [2]:
path = "../final_assignment_public/Telecom_customer_churn.csv"
df = pd.read_csv( path )

## Data Preprocessing

### Categorical data

In [3]:
# カテゴリカル変数を表示
x_cat = df[df.select_dtypes( include=object ).columns].columns

# カテゴリカル変数を落とす(理解不能なため)
df = df.drop( columns=x_cat )
df

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,opk_dat_Mean,mou_opkv_Mean,mou_opkd_Mean,drop_blk_Mean,attempt_Mean,complete_Mean,callfwdv_Mean,callwait_Mean,churn,months,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,forgntvl,eqpdays,Customer_ID
0,23.9975,219.25,22.500,0.2475,0.00,0.0,0.0,0.0,0.0,-157.25,-18.9975,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.000000,42.333333,0.0,45.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,90.643333,0.000000,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.000000,132.600000,0.000000,24.000000,0.0,55.220000,0.0,1.333333,52.333333,45.000000,0.0,0.333333,1,61,2,1,1652,4228.00000,1504.62,1453.44,4085.00,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,149.98999,2.0,2.0,0.0,0.0,15.0,1.0,4.0,3.0,0.0,361.0,1000001
1,57.4925,482.75,37.425,0.2475,22.75,9.1,9.1,0.0,0.0,532.25,50.9875,8.333333,0.0,1.000000,0.0,61.333333,0.0,263.333333,0.000000,69.000000,0.0,193.333333,0.000000,1.666667,6.333333,5.463333,53.000000,0.333333,189.396667,0.000000,55.280000,46.333333,24.216667,6.333333,3.696667,83.666667,0.000000,75.333333,0.000000,157.000000,0.0,169.343333,0.0,9.333333,263.333333,193.333333,0.0,5.666667,0,56,1,1,14654,26400.00000,2851.68,2833.88,26367.00,14624,51.53,479.40,265.89,305,158,40,477.0,275.0,48.0,NaN,7.0,6.0,1.0,1.0,1.0,1.0,5.0,1.0,0.0,240.0,1000002
2,16.9900,10.25,16.990,0.0000,0.00,0.0,0.0,0.0,0.0,-4.25,0.0000,0.333333,0.0,0.000000,0.0,2.666667,0.0,9.000000,0.000000,0.333333,0.0,6.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,5.426667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,5.193333,0.000000,1.000000,0.0,0.233333,0.0,0.333333,9.000000,6.000000,0.0,0.000000,1,58,1,1,7903,24385.05333,2155.91,1934.47,24303.05,7888,34.54,433.98,140.86,12,7,17,11.0,6.0,17.0,29.98999,2.0,1.0,0.0,0.0,7.0,2.0,5.0,2.0,0.0,1504.0,1000003
3,38.0000,7.50,38.000,0.0000,0.00,0.0,0.0,0.0,0.0,-1.50,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,3.666667,0.000000,1.333333,0.0,3.666667,0.000000,0.000000,0.000000,0.000000,1.333333,0.000000,8.410000,0.000000,0.413333,0.333333,0.256667,0.000000,0.000000,1.333333,0.000000,3.380000,0.000000,3.666667,0.0,5.450000,0.0,0.000000,3.666667,3.666667,0.0,0.000000,0,60,1,1,1502,3065.00000,2000.90,1941.81,3035.00,1479,40.45,63.23,30.81,8,3,38,50.0,25.0,40.0,29.98999,1.0,1.0,0.0,0.0,6.0,4.0,6.0,1.0,0.0,1812.0,1000004
4,55.2300,570.50,71.980,0.0000,0.00,0.0,0.0,0.0,0.0,38.50,0.0000,9.666667,0.0,0.666667,0.0,77.000000,0.0,222.333333,0.000000,94.666667,0.0,137.000000,0.000000,8.666667,15.000000,11.076667,66.000000,0.000000,285.233333,0.000000,106.330000,14.666667,10.816667,0.666667,0.366667,97.333333,0.000000,173.476667,0.000000,90.333333,0.0,218.086667,0.0,10.333333,222.333333,137.000000,0.0,0.000000,0,57,1,1,4485,14028.00000,2181.12,2166.48,13965.00,4452,38.69,249.38,79.50,558,191,55,586.0,196.0,80.0,149.98999,6.0,4.0,0.0,0.0,5.0,1.0,6.0,1.0,0.0,434.0,1000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,0.0,0.000000,0.0,7.666667,0.0,54.666667,0.000000,3.000000,0.0,45.333333,

### Numerical data

In [4]:
# 数値変数を確認
numerical = df[df.select_dtypes( exclude=object ).columns]
x_num = numerical.columns

# 欠損値を0で埋める
df[x_num] = numerical.fillna(0)
df

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,opk_dat_Mean,mou_opkv_Mean,mou_opkd_Mean,drop_blk_Mean,attempt_Mean,complete_Mean,callfwdv_Mean,callwait_Mean,churn,months,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,forgntvl,eqpdays,Customer_ID
0,23.9975,219.25,22.500,0.2475,0.00,0.0,0.0,0.0,0.0,-157.25,-18.9975,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.000000,42.333333,0.0,45.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,90.643333,0.000000,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.000000,132.600000,0.000000,24.000000,0.0,55.220000,0.0,1.333333,52.333333,45.000000,0.0,0.333333,1,61,2,1,1652,4228.00000,1504.62,1453.44,4085.00,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,149.98999,2.0,2.0,0.0,0.0,15.0,1.0,4.0,3.0,0.0,361.0,1000001
1,57.4925,482.75,37.425,0.2475,22.75,9.1,9.1,0.0,0.0,532.25,50.9875,8.333333,0.0,1.000000,0.0,61.333333,0.0,263.333333,0.000000,69.000000,0.0,193.333333,0.000000,1.666667,6.333333,5.463333,53.000000,0.333333,189.396667,0.000000,55.280000,46.333333,24.216667,6.333333,3.696667,83.666667,0.000000,75.333333,0.000000,157.000000,0.0,169.343333,0.0,9.333333,263.333333,193.333333,0.0,5.666667,0,56,1,1,14654,26400.00000,2851.68,2833.88,26367.00,14624,51.53,479.40,265.89,305,158,40,477.0,275.0,48.0,0.00000,7.0,6.0,1.0,1.0,1.0,1.0,5.0,1.0,0.0,240.0,1000002
2,16.9900,10.25,16.990,0.0000,0.00,0.0,0.0,0.0,0.0,-4.25,0.0000,0.333333,0.0,0.000000,0.0,2.666667,0.0,9.000000,0.000000,0.333333,0.0,6.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,5.426667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,5.193333,0.000000,1.000000,0.0,0.233333,0.0,0.333333,9.000000,6.000000,0.0,0.000000,1,58,1,1,7903,24385.05333,2155.91,1934.47,24303.05,7888,34.54,433.98,140.86,12,7,17,11.0,6.0,17.0,29.98999,2.0,1.0,0.0,0.0,7.0,2.0,5.0,2.0,0.0,1504.0,1000003
3,38.0000,7.50,38.000,0.0000,0.00,0.0,0.0,0.0,0.0,-1.50,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,3.666667,0.000000,1.333333,0.0,3.666667,0.000000,0.000000,0.000000,0.000000,1.333333,0.000000,8.410000,0.000000,0.413333,0.333333,0.256667,0.000000,0.000000,1.333333,0.000000,3.380000,0.000000,3.666667,0.0,5.450000,0.0,0.000000,3.666667,3.666667,0.0,0.000000,0,60,1,1,1502,3065.00000,2000.90,1941.81,3035.00,1479,40.45,63.23,30.81,8,3,38,50.0,25.0,40.0,29.98999,1.0,1.0,0.0,0.0,6.0,4.0,6.0,1.0,0.0,1812.0,1000004
4,55.2300,570.50,71.980,0.0000,0.00,0.0,0.0,0.0,0.0,38.50,0.0000,9.666667,0.0,0.666667,0.0,77.000000,0.0,222.333333,0.000000,94.666667,0.0,137.000000,0.000000,8.666667,15.000000,11.076667,66.000000,0.000000,285.233333,0.000000,106.330000,14.666667,10.816667,0.666667,0.366667,97.333333,0.000000,173.476667,0.000000,90.333333,0.0,218.086667,0.0,10.333333,222.333333,137.000000,0.0,0.000000,0,57,1,1,4485,14028.00000,2181.12,2166.48,13965.00,4452,38.69,249.38,79.50,558,191,55,586.0,196.0,80.0,149.98999,6.0,4.0,0.0,0.0,5.0,1.0,6.0,1.0,0.0,434.0,1000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0000,0.00,0.000,0.0000,0.00,0.0,0.0,0.0,0.0,0.00,0.0000,2.000000,0.0,0.000000,0.0,7.666667,0.0,54.666667,0.000000,3.000

## Train Test Split

In [5]:
df_train, df_test = train_test_split( df, test_size=0.2, random_state=4 )

print( 'df_train: {}'.format( len(df_train) ) )
print( 'df_test : {}'.format( len(df_test) ) )

df_train: 80000
df_test : 20000


In [6]:
# trainデータを説明変数(X_train)と目的変数(y_train)に分割
X_train = df_train.drop( 'churn', axis=1 )
y_train = df_train['churn']
 
# testデータを説明変数(X_test)と目的変数(y_test)に分割
X_test  = df_test.drop( 'churn', axis=1 )
y_test  = df_test['churn']

## Train & Predict

In [53]:
# LightGBM model
model = LGBMClassifier(
        boosting_type='gbdt', 
        class_weight='balanced',
        objective='binary',
        metric='auc',
        max_depth=10,
        num_leaves=100
        # num_iteration=100,
        # num_threads=1,
)

In [54]:
# 学習
model.fit( X_train, y_train )

LGBMClassifier(class_weight='balanced', max_depth=10, metric='auc',
               num_leaves=100, objective='binary')

In [55]:
# 予測
y_pred = model.predict( X_test )
y_pred_prob = model.predict_proba( X_test ) # テストデータのクラス予測確率を返す

## Evaluate the model

In [56]:
# 真値と予測値の表示
df_pred = pd.DataFrame( {'churn_GT':y_test, 'churn_pred':y_pred} )
display( df_pred )

,churn_GT,churn_pred
60645,1,1
84599,1,1
59801,0,0
90055,0,0
94213,1,0
...,...,...
3081,0,0
98897,0,0
69223,0,0
34419,1,1


In [69]:
GT0   = df_pred['churn_GT'] == 0
pred0 = df_pred['churn_pred'] == 0

GT1   = df_pred['churn_GT'] == 1
pred1 = df_pred['churn_pred'] == 1

print( 'GT0: {},\tpred0: {}'.format( GT0.sum(), pred0.sum() ) )
print( 'GT1: {},\tpred1: {}'.format( GT1.sum(), pred1.sum() ) )

# acc: 正答率
acc = accuracy_score( y_test, y_pred )
print( '\nAcc:', acc )

GT0: 10030,	pred0: 9693
GT1: 9970,	pred1: 10307

Acc: 0.63655


## 仮説検証

### 解約者と予測された人のインデックスを取得

In [80]:
target_idx = df_pred[df_pred['churn_pred'] == 1].index
target_idx

Int64Index([60645, 84599, 71675, 47096, 80437, 30836, 55530, 61443, 35372,
            58169,
            ...
             8104,   660, 30207, 20020,  3114, 49288, 54486, 18726, 15956,
            34419],
           dtype='int64', length=10307)

### 「解約者と予測された人にのみ」仮説を適用

In [117]:
# 解約者と予測された人のdfを取得
df_target = df.iloc[target_idx]

# 1. 端末の使用日数を「0」に設定
df_target['eqpdays'] = 0

# 2. 端末価格を，「非解約者の平均+標準偏差」に設定
hnd_price_of_new_eqp = round( df.groupby('churn')['hnd_price'].mean()[0] ) + round( df.groupby('churn')['hnd_price'].std()[0] )
df_target['hnd_price'] = hnd_price_of_new_eqp

# 3. 月平均の使用時間の変化率を「0」に設定
df_target['change_mou'] = 0

# 4. 月平均の収入に，「端末料金の値下げ分」を追加
df_target['change_rev'] = df_target['change_rev'] + hnd_price_of_new_eqp*0.2

# # 「解約者のみ」請求額を2割引
# df['rev_Mean'] = df['rev_Mean'] * 0.8
# df['totmrc_Mean'] = df['totmrc_Mean'] * 0.8

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,opk_dat_Mean,mou_opkv_Mean,mou_opkd_Mean,drop_blk_Mean,attempt_Mean,complete_Mean,callfwdv_Mean,callwait_Mean,churn,months,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,forgntvl,eqpdays,Customer_ID
60645,41.243116,1213.50,7.864320,0.2475,356.75,124.8625,124.8625,0.0,0.1950,0,80.1700,4.000000,0.0,10.000000,0.0,63.000000,0.0,380.000000,0.0,163.666667,0.0,304.000000,0.0,0.000000,0.000000,0.000000,66.666667,0.000000,481.673333,0.0,410.176667,105.666667,94.690000,36.000000,85.626667,224.000000,0.0,375.986667,0.0,242.666667,0.0,515.863333,0.0,14.000000,380.000000,304.000000,0.0,10.000000,1,14,1,1,4386,12392.0,1294.85,1234.86,12383.0,4383,94.99,952.54,337.15,1175,433,142,1127.0,394.0,129.0,169,1.0,1.0,0.0,0.0,8.0,6.0,9.0,2.0,1.0,0,1060646
84599,29.084877,780.25,11.476664,0.2475,180.25,69.3250,69.3250,0.0,0.0975,0,24.4300,11.333333,0.0,7.666667,0.0,69.000000,0.0,317.000000,0.0,94.333333,0.0,233.333333,0.0,2.000000,6.666667,5.806667,41.666667,0.000000,356.103333,0.0,172.430000,35.333333,29.220000,2.333333,5.856667,212.333333,0.0,347.413333,0.0,109.666667,0.0,181.120000,0.0,19.000000,317.000000,233.333333,0.0,2.333333,1,13,2,1,3668,9866.0,983.74,948.12,9852.0,3663,86.19,895.64,333.00,724,312,114,907.0,341.0,106.0,169,1.0,1.0,0.0,0.0,7.0,6.0,7.0,0.0,0.0,0,1084600
71675,20.538982,709.50,16.055009,0.0000,64.50,19.3500,19.3500,0.0,1.5050,0,4.6300,3.000000,0.0,1.000000,0.0,14.000000,0.0,116.666667,0.0,59.666667,0.0,100.000000,0.0,5.666667,44.333333,41.443333,28.333333,0.000000,368.073333,0.0,249.786667,15.000000,17.766667,1.000000,1.046667,102.666667,0.0,448.296667,0.0,44.333333,0.0,169.563333,0.0,4.000000,116.666667,100.000000,0.0,1.666667,0,12,1,1,1204,5776.0,833.63,773.65,5511.0,1188,70.33,501.00,108.00,688,149,88,553.0,129.0,70.0,169,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1071676
47096,40.616591,619.75,39.318979,4.9500,0.00,0.0000,0.0000,0.0,0.0000,0,32.8100,3.000000,0.0,30.000000,0.0,19.666667,0.0,326.666667,0.0,56.666667,0.0,275.666667,0.0,0.000000,0.000000,0.000000,28.333333,0.333333,321.226667,0.0,85.913333,32.666667,23.353333,2.666667,4.203333,300.333333,0.0,362.080000,0.0,28.000000,0.0,45.060000,0.0,33.000000,326.666667,275.666667,0.0,7.666667,1,18,1,1,7895,18018.0,2919.02,2739.04,17976.0,7879,161.12,1057.41,463.47,689,349,155,766.0,372.0,155.0,169,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1047097
80437,52.834468,1447.50,22.988063,2.4750,402.50,120.7500,120.7500,0.0,0.6300,0,-80.6275,39.666667,0.0,2.000000,0.0,80.666667,0.0,348.000000,0.0,229.666667,0.0,242.333333,0.0,0.000000,0.000000,0.000000,118.000000,0.000000,529.350000,0.0,461.683333,70.000000,88.230000,40.000000,118.263333,344.000000,0.0,833.133333,0.0,72.000000,0.0,157.900000,0.0,41.666667,348.000000,242.333333,0.0,12.666667,0,9,3,3,2774,9786.0,1214.07,1183.75,9786.0,2774,147.97,1223.25,346.75,1608,467,239,1330.0,381.0,167.0,169,1.0,1.0,0.0,0.0,7.0,2.0,6.0,0.0,0.0,0,1080438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49288,15.318385,673.50,10.483794,0.7425,51.75,20.7000,20.7000,0.0,0.0000,0,-23.6450,7.666667,0.0,0.000000,0.0,6.666667,0.0,244.3333

In [122]:
print( 'hnd_price_of_new_eqp:', hnd_price_of_new_eqp )
print( 'hnd_price_of_new_eqp*0.2:', hnd_price_of_new_eqp*0.2 )

hnd_price_of_new_eqp: 169
hnd_price_of_new_eqp*0.2: 33.800000000000004


In [118]:
df.iloc[target_idx] = df_target
df

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,opk_dat_Mean,mou_opkv_Mean,mou_opkd_Mean,drop_blk_Mean,attempt_Mean,complete_Mean,callfwdv_Mean,callwait_Mean,churn,months,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,forgntvl,eqpdays,Customer_ID
0,6.290801,219.25,5.898240,0.2475,0.00,0.0,0.0,0.0,0.0,-157.25,-18.9975,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.000000,42.333333,0.0,45.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,90.643333,0.000000,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.000000,132.600000,0.000000,24.000000,0.0,55.220000,0.0,1.333333,52.333333,45.000000,0.0,0.333333,1,61,2,1,1652,4228.00000,1504.62,1453.44,4085.00,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,149.98999,2.0,2.0,0.0,0.0,15.0,1.0,4.0,3.0,0.0,361.0,1000001
1,15.071314,482.75,9.810739,0.2475,22.75,9.1,9.1,0.0,0.0,532.25,50.9875,8.333333,0.0,1.000000,0.0,61.333333,0.0,263.333333,0.000000,69.000000,0.0,193.333333,0.000000,1.666667,6.333333,5.463333,53.000000,0.333333,189.396667,0.000000,55.280000,46.333333,24.216667,6.333333,3.696667,83.666667,0.000000,75.333333,0.000000,157.000000,0.0,169.343333,0.0,9.333333,263.333333,193.333333,0.0,5.666667,0,56,1,1,14654,26400.00000,2851.68,2833.88,26367.00,14624,51.53,479.40,265.89,305,158,40,477.0,275.0,48.0,0.00000,7.0,6.0,1.0,1.0,1.0,1.0,5.0,1.0,0.0,240.0,1000002
2,4.453827,10.25,4.453827,0.0000,0.00,0.0,0.0,0.0,0.0,-4.25,0.0000,0.333333,0.0,0.000000,0.0,2.666667,0.0,9.000000,0.000000,0.333333,0.0,6.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,5.426667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,5.193333,0.000000,1.000000,0.0,0.233333,0.0,0.333333,9.000000,6.000000,0.0,0.000000,1,58,1,1,7903,24385.05333,2155.91,1934.47,24303.05,7888,34.54,433.98,140.86,12,7,17,11.0,6.0,17.0,29.98999,2.0,1.0,0.0,0.0,7.0,2.0,5.0,2.0,0.0,1504.0,1000003
3,9.961472,7.50,9.961472,0.0000,0.00,0.0,0.0,0.0,0.0,-1.50,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,3.666667,0.000000,1.333333,0.0,3.666667,0.000000,0.000000,0.000000,0.000000,1.333333,0.000000,8.410000,0.000000,0.413333,0.333333,0.256667,0.000000,0.000000,1.333333,0.000000,3.380000,0.000000,3.666667,0.0,5.450000,0.0,0.000000,3.666667,3.666667,0.0,0.000000,0,60,1,1,1502,3065.00000,2000.90,1941.81,3035.00,1479,40.45,63.23,30.81,8,3,38,50.0,25.0,40.0,29.98999,1.0,1.0,0.0,0.0,6.0,4.0,6.0,1.0,0.0,1812.0,1000004
4,14.478213,570.50,18.869125,0.0000,0.00,0.0,0.0,0.0,0.0,38.50,0.0000,9.666667,0.0,0.666667,0.0,77.000000,0.0,222.333333,0.000000,94.666667,0.0,137.000000,0.000000,8.666667,15.000000,11.076667,66.000000,0.000000,285.233333,0.000000,106.330000,14.666667,10.816667,0.666667,0.366667,97.333333,0.000000,173.476667,0.000000,90.333333,0.0,218.086667,0.0,10.333333,222.333333,137.000000,0.0,0.000000,0,57,1,1,4485,14028.00000,2181.12,2166.48,13965.00,4452,38.69,249.38,79.50,558,191,55,586.0,196.0,80.0,149.98999,6.0,4.0,0.0,0.0,5.0,1.0,6.0,1.0,0.0,434.0,1000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.000000,0.00,0.000000,0.0000,0.00,0.0,0.0,0.0,0.0,0.00,0.0000,2.000000,0.0,0.000000,0.0,7.666667,0.0,5

### 再度分割

In [119]:
df_train, df_test = train_test_split( df, test_size=0.2, random_state=4 )

print( 'df_train: {}'.format( len(df_train) ) )
print( 'df_test : {}'.format( len(df_test) ) )

# trainデータを説明変数(X_train)と目的変数(y_train)に分割
X_train = df_train.drop( 'churn', axis=1 )
y_train = df_train['churn']
 
# testデータを説明変数(X_test)と目的変数(y_test)に分割
X_test  = df_test.drop( 'churn', axis=1 )
y_test  = df_test['churn']

df_train: 80000
df_test : 20000


### 再度学習&予測

In [120]:
# 学習
model.fit( X_train, y_train )

# 予測
y_pred = model.predict( X_test )
y_pred_prob = model.predict_proba( X_test ) # テストデータのクラス予測確率を返す

# 真値と予測値の表示
df_pred = pd.DataFrame( {'churn_GT':y_test, 'churn_pred':y_pred} )
display( df_pred )

,churn_GT,churn_pred
60645,1,0
84599,1,1
59801,0,0
90055,0,0
94213,1,0
...,...,...
3081,0,0
98897,0,0
69223,0,0
34419,1,0


### モデルの再評価

In [121]:
GT0   = df_pred['churn_GT'] == 0
pred0 = df_pred['churn_pred'] == 0

GT1   = df_pred['churn_GT'] == 1
pred1 = df_pred['churn_pred'] == 1

print( 'GT0: {},\tpred0: {}'.format( GT0.sum(), pred0.sum() ) )
print( 'GT1: {},\tpred1: {}'.format( GT1.sum(), pred1.sum() ) )

# acc: 正答率
acc = accuracy_score( y_test, y_pred )
print( '\nAcc:', acc )

GT0: 10030,	pred0: 17548
GT1: 9970,	pred1: 2452

Acc: 0.5492
